In [ ]:
import os
import sys
import json
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from PIL import Image
from torch.utils.data import random_split, DataLoader, ConcatDataset
import torch.nn as nn
from torchvision.models import efficientnet_b0

# ─────────────────────────────────────────────────────────────────────────────
# 1. Повторякем подготовку данных 
# ─────────────────────────────────────────────────────────────────────────────
seminar_path = "/home/alexskv/seminar_2"
seminar_path = Path(seminar_path)
data_path = f"{seminar_path}/data"
sys.path.insert(0, f"{seminar_path}/core")
print(sys.path)

from dataset import SimpleCocoDataset
from trainers import SimpleClassificationTrainer

pathology_ids = [i for i in range(6, 27) if i != 15]   # 6‑26, кроме 15

out_classes = [ {"id": 1, "name": "Патология", "summable_masks": pathology_ids, "subtractive_masks": []}]

base_names = [
    "Правое лёгкое", "Левое лёгкое", "Контуры сердца", "Купола диафрагмы и нижележащая область",
    "Сложный случай", "нельзя составить заключение", "Иная патология", "Гидроторакс",
    "Легочно-венозная гипертензия 2 стадии и выше", "Пневмоторакс", "Доброкачественное новообразование",
    "Перелом ребра свежий", "Буллезное вздутие, тонкостенная киста", "Рак лёгкого (включая дорожку к корню при наличии)",
    "Кардиомегалия (отмечается всё сердце, как патология)", "Интерстициальная пневмония.",
    "Метастатическое поражение лёгких", "Полость с уровнем жидкости", "Грыжа пищевого отверстия диафрагмы",
    "Спавшийся сегмент лёгкого при ателектазе", "Инфильтративный туберкулёз",
    "Пневмония. В том числе сегментарная и полисегментарная", "Область распада, деструкции тканей лёгкого",
    "Участок пневмофиброза", "Кальцинаты. Каждый кальцинат выделяется отдельным контуром",
    "Консолидированный перелом ребра"
]

base_classes = [{"id": i+1, "name": name} for i, name in enumerate(base_names)]


#Настроим параметры даталодера
batch_size = 64
resize = (512, 512)


data_roots   = {p.parent.parent for p in (seminar_path / "data").rglob("annotations/instances_default.json")}

print(f"Найдено {len(data_roots)} датасетов:", *data_roots, sep="\n  ")


datasets = [SimpleCocoDataset(str(d),
                              base_classes,
                              out_classes,
                              resize=resize)
            for d in sorted(data_roots)]

full_ds  = ConcatDataset(datasets)


In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# 2. Разделение на train/val
# ─────────────────────────────────────────────────────────────────────────────
val_percent = 0.2
val_size = int(len(full_ds) * val_percent)
train_size = len(full_ds) - val_size
train_ds, val_ds = random_split(full_ds, [train_size, val_size])

train_loader = DataLoader(train_ds,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=4)

val_loader = DataLoader(val_ds,
                        batch_size=batch_size,
                        shuffle=False,
                        num_workers=4)





In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# 3. Создаем модель EfficientNet адаптированная под 1 выход
# ─────────────────────────────────────────────────────────────────────────────
class MonoEfficientNet(nn.Module):
    def __init__(self):
        super().__init__()
        model = efficientnet_b0(weights=None)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, 1)
        self.backbone = model

    def forward(self, x):
        if x.shape[1] == 1:
            x = x.repeat(1, 3, 1, 1)  # 1 канал → 3
        return self.backbone(x)

model = MonoEfficientNet()

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# 4. Запуск обучения
# ─────────────────────────────────────────────────────────────────────────────
device = torch.device("cuda:1")

trainer = SimpleClassificationTrainer(
    model=model,
    classes = out_classes,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    epochs=10,
    exp_name="efficientnet_binary_cls"
)

trainer.train()